Convert shapefile to geojson

In [14]:
!shp2json natural-earth-data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp -o world/countries.json

Split up the geojson file so that each line ends with a `\n` character

In [15]:
!ndjson-split 'd.features' \
    < world/countries.json \
    > world/countries.ndjson

In [16]:
!wc -l world/countries.ndjson

     177 world/countries.ndjson


In [17]:
!rm world/countries.json

Add a `Code` key for each country, so that it can be used for the join and reduce the amount of properties for each country, to just the name `NAME_EN`, country code `ISO_A3` and region `REGION_UN` and `SUBREGION`

In [18]:
!ndjson-map 'd.Code = d.properties.ISO_A3, d' < world/countries.ndjson \
    | ndjson-map 'd.properties = { Name:d.properties.NAME_EN, Code:d.properties.ISO_A3, region:d.properties.REGION_UN, subregion:d.properties.SUBREGION }, d' \
    > world/temp.ndjson

In [19]:
!rm world/countries.ndjson

In [20]:
!mv world/temp.ndjson world/countries.ndjson

Turn the marlaria-death-rates.csv into a NDJSON file

In [21]:
!csv2json -n \
    < data/malaria-death-rates-by-year.csv \
    > world/malaria-death-rates-by-year.ndjson

Join the deaths data to the world map data. Make this a left outer join, so that each country is kept, even if they don't have any malaria data.

In [22]:
!ndjson-join --left 'd.Code' \
    world/countries.ndjson \
    world/malaria-death-rates-by-year.ndjson \
    > world/temp.ndjson

In [23]:
!rm world/countries.ndjson
!mv world/temp.ndjson world/countries.ndjson

This results in a file with two json objects per line. Add the second object as a 'deaths' to the first objects 'properties'
To get rid of the second object use ndjson-map and Object.assign

In [24]:
!ndjson-map 'd[0].properties.deaths = d[1], d' < world/countries.ndjson \
    | ndjson-map 'Object.assign(d[0])' \
    > world/temp.ndjson

In [25]:
!rm world/countries.ndjson
!mv world/temp.ndjson world/countries.ndjson

Convert back from ndjson

In [26]:
!ndjson-reduce < world/countries.ndjson \
    | ndjson-map '{type: "FeatureCollection", features: d}' \
    > world/countries.json

In [27]:
!rm world/countries.ndjson
!rm world/malaria-death-rates-by-year.ndjson

Convert from from geojson to topojson

In [28]:
!geo2topo world/countries.json > world/countries-topo.json

Simplify the topojson to reduce file size 

In [30]:
!toposimplify -p 0.01 -f < world/countries-topo.json > world/temp.json

In [31]:
!rm world/countries-topo.json
!mv world/temp.json world/countries-topo.json